In [5]:
import pandas as pd
import os
import pandas as pd
import torch
import pickle
import nltk
from tqdm import tqdm
import random

In [2]:
ovo_path = "../CODE_POST_OVERALL-EMBEDDINGS_DATA_V3.pkl"

# 1. Load OVO_data once
with open(ovo_path, 'rb') as f:
    OVO_data = pickle.load(f)

# 2. Extract raw questions, question embeddings, and answer_sentences_list once

# question_embeddings = [item['question_embedding'] for item in OVO_data]
# raw_accepted_answers = [item['raw_accepted_answer'] for item in OVO_data]
# answer_sentences_list = [item['answer_sentences'] for item in OVO_data]

/home/21fl11/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [4]:
raw_questions = [item['raw_question'] for item in OVO_data]
raw_accepted_answers = [item['raw_accepted_answer'] for item in OVO_data]

In [6]:
# Ensure the lists are the same length
assert len(raw_questions) == len(raw_accepted_answers)

# Sample 385 unique indices
sample_indices = random.sample(range(len(raw_questions)), 385)

# Create sampled lists
sampled_questions = [raw_questions[i] for i in sample_indices]
sampled_answers = [raw_accepted_answers[i] for i in sample_indices]

# Compose into a DataFrame
sampled_df = pd.DataFrame({
    'Question': sampled_questions,
    'Accepted Answer': sampled_answers
})

In [7]:
sampled_df

,Question,Accepted Answer
0,Can't get grunt-browser-sync in Cloud9-IDE to ...,<p>You are configuring BrowserSync to proxy fr...
1,optimize a string query with pandas. large data,<p><strong>Setup</strong></p>\n\n<pre><code>df...
2,Generate encoding String according to creation...,<p>What you want is <code>SELECT MAX(id)</code...
3,How to convert ResultSet into a useful integer?,<p>A <code>ResultSet</code> has a series of <c...
4,Inheritance-aware class to value map to replac...,<p>A map will do. If you want class inheritanc...
...,...,...
380,Bool query with fuzziness in elastic search ja...,"<p>The problem is, that method <code>fuziness(..."
381,Sending data to specific user via node.js and ...,<p>You need the id of the socket (socket.id) y...
382,Join the same argument mulitple times from list,<p>Looks like you just have a bracket in the w...
383,Unable to execute onClick javascript selenium ...,<p>I managed to get this done by reverting bac...


In [ ]:
# sampled_df.to_csv(file_path, index=False)

In [8]:
from openai import OpenAI
import pandas as pd
import json
import time 
from tqdm import tqdm  

In [ ]:
client = OpenAI(api_key = '')

In [10]:
def build_paraphrase_prompt(question_title):
    prompt = f"""
Paraphrase the following question: 
{question_title}
"""
    return prompt.strip()


In [ ]:

paraphrase_text_list = []


for idx, row in tqdm(sampled_df.iterrows(), total=len(sampled_df)):
    
    question_title = row["Question"]

    prompt_text = build_paraphrase_prompt(
        question_title,
    )
    
    try:
        
        response = client.chat.completions.create(

            model="gpt-4o",  
            messages=[
                {"role": "system", "content": "You are an expert of Java and Python. Help me paraphrase the input question to similar question title that people never asked"},
                {"role": "user", "content": prompt_text}
            ],
            temperature=0.7  
        )
        
        paraphrase_text = response.choices[0].message.content

        paraphrase_text_list.append(paraphrase_text)
        time.sleep(0.5)
    except Exception as e:

        print(f"OpenAI API error on row {idx}: {e}")
        paraphrase_text_list.append("Error")




100%|██████████| 385/385 [07:19<00:00,  1.14s/it]


In [12]:
sampled_df['Paraphrased Question'] = paraphrase_text_list

In [13]:
sampled_df

,Question,Accepted Answer,Paraphrased Question
0,Can't get grunt-browser-sync in Cloud9-IDE to ...,<p>You are configuring BrowserSync to proxy fr...,Having trouble setting up grunt-browser-sync i...
1,optimize a string query with pandas. large data,<p><strong>Setup</strong></p>\n\n<pre><code>df...,Enhancing string search efficiency using panda...
2,Generate encoding String according to creation...,<p>What you want is <code>SELECT MAX(id)</code...,Create a string representation based on the se...
3,How to convert ResultSet into a useful integer?,<p>A <code>ResultSet</code> has a series of <c...,What methods can be used to transform a Result...
4,Inheritance-aware class to value map to replac...,<p>A map will do. If you want class inheritanc...,Class-to-value mapping that recognizes inherit...
...,...,...,...
380,Bool query with fuzziness in elastic search ja...,"<p>The problem is, that method <code>fuziness(...",How to implement a fuzzy boolean query using t...
381,Sending data to specific user via node.js and ...,<p>You need the id of the socket (socket.id) y...,Transmitting information to an individual user...
382,Join the same argument mulitple times from list,<p>Looks like you just have a bracket in the w...,Combine the identical argument several times f...
383,Unable to execute onClick javascript selenium ...,<p>I managed to get this done by reverting bac...,Experiencing difficulties with executing JavaS...


In [14]:
sampled_df.to_csv('Sythetic_old_question.csv', index=False)